In [1]:
import csv
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Convolution2D, MaxPooling2D, Lambda, ELU, Dropout, Cropping2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def get_lines_csv(data_path):
    
    lines=[]
    with open(data_path +'/'+ 'driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)
    
    lines = lines[1:]       
    return lines


def get_images(data_path):
    
    dir_location = [x[0] for x in os.walk(data_path)]
    csv_file_dir = list(filter(lambda y: os.path.isfile(y + '/driving_log.csv'), dir_location))
        
    center_image=[]
    left_image=[]
    right_image=[]
    steering_angle=[]
    
    for path in csv_file_dir:
        lines=get_lines_csv(path)
        center_im=[]
        left_im=[]
        right_im=[]
        steer=[]
        
        for line in lines:
            center_im.append(path + '/' + line[0])
            left_im.append(path + '/' + line[1])
            right_im.append(path + '/' + line[2])
            steer.append(float(line[3]))
            
            center_image.extend(center_im)
            left_image.extend(left_im)
            right_image.extend(right_im)
            steering_angle.extend(steer)

    return (center_image, left_image, right_image, steering_angle)

def join_imgs(center_img, left_img, right_img, steering_ang, corr=0.2):
    
    final_img_path=[]
    final_steering=[]
    
    final_img_path.extend(center_img)
    final_img_path.extend(left_img)
    final_img_path.extend(right_img)
    final_steering.extend(steering_ang)
    final_steering.extend([x+corr for x in steering_ang])
    final_steering.extend([x-corr for x in steering_ang])
    
    return final_img_path, final_steering

Using TensorFlow backend.


In [ ]:
center_img, left_img, right_img, steering_ang= get_images('Data')
final_img_path, final_steering = join_imgs(center_img, left_img, right_img, steering_ang)


images=[]
measurement=[]

for i in range(len(final_img_path)):
    source_path= final_img_path[i]
    filename = source_path.split('\\')[-1]
    current_path = './Data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)

for j in range(len(final_steering)):
    measurement.append(final_steering[j])

X_train=np.array(images)
y_train=np.array(measurement)

print('Number of Train samples: {}'.format(len(X_train)))
print('Number of Train Labels: {}' .format(len(y_train)))

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train ,y_train , validation_split=0.2, shuffle=True, nb_epoch=8)
model.save('model.h5')
print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])